## **Experiments**

In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [ ]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-4o"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding = embd,
            persist_path = persist_path,
            serializer = "parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path = "https://docs.smith.langchain.com/sitemap.xml", continue_on_failure = True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size = 500, chunk_overlap = 0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents = doc_splits,
        embedding = embd,
        persist_path = persist_path,
        serializer = "parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type = "chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type = "chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type = "llm",
    metadata = {
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model = MODEL_NAME,
        messages = messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


### **Experiment**

In [7]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "RAG Application Golden Dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data = dataset_name,
    evaluators = [is_concise_enough],
    experiment_prefix = "gpt-4o"
)

View the evaluation results for experiment: 'gpt-4o-1128dfdf' at:
https://smith.langchain.com/o/0d21f241-5c3d-4a95-af25-5842839689d3/datasets/32edf740-86da-490e-9e2f-d0faf30d18f9/compare?selectedSessions=9d4ff537-0960-4127-a9cd-8a65e3e97aa0




13it [00:31,  2.41s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I add tags to a Trace?,You can add tags to a trace in LangSmith by ei...,None,You can add tags to a trace by annotating it e...,1,2.563425,9e30ed8a-08aa-4d87-9847-b0c67da920c4,ffe8aab7-4e94-4fe4-a01d-a307fabd59b0
1,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith with LangChain,...",None,Setting up tracing to LangSmith while using La...,1,2.883592,fee59a4e-f5d8-4f85-acfb-8433bebb6403,cc43f107-520e-4150-aba3-4166cd64df50
2,"{\n ""Question"": ""Is there a Javascript LangSmi...","Yes, there is a JavaScript LangSmith SDK avail...",None,Yes there is a JavaScript LangSmith SDK,1,1.082494,d4b8c798-bafd-441e-8139-b76900b8cf9d,90fd78bf-5f05-4088-a820-cf9cd22dff77
3,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,2.002966,070ce5f5-f180-4c04-8dfb-705f8201d79a,6ae08101-c982-4cd2-bf20-7a977a123e49
4,What is LangSmith used for in three sentences?,LangSmith is a platform for building productio...,None,LangSmith is a platform designed for the devel...,1,5.193034,0a303fe1-e0ec-4e4f-b6db-ecfcb08f8d6c,49c4d014-82e3-402e-9675-3336b6ad7e5b
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,0,2.705394,0d731f1e-c7e9-4b10-bb80-4ab72cea0015,316c6870-7145-40dd-8cb7-23237a640e49
6,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation.",None,"Yes, LangSmith supports offline evaluation thr...",1,1.262527,236f4c49-773a-4bf9-b31f-790d38b1761a,089e98fa-017d-45d4-abbe-906fd8bfa3aa
7,How do I create user feedback with the LangSmi...,You can create user feedback with the LangSmit...,None,To create user feedback with the LangSmith SDK...,1,2.758672,6757b357-9aa0-48f9-96c5-b829c57662d9,515b4f00-f462-4972-969a-989ca929b9a7
8,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents,...",None,"Yes, LangSmith can be used to evaluate agents....",1,2.127916,6cccd322-eec7-45b2-acd2-84ccfdb5e0ba,50e049c0-3521-4832-a77b-97f9a1d648c8
9,How do I pass metadata in with @traceable?,To pass metadata with the `@traceable` decorat...,None,You can pass metadata with the @traceable deco...,1,2.194648,7b6dddd4-f2bf-4aaa-b1fd-8098e73e4c21,169de5ce-e126-40a5-a07f-3064572b3bbc


### **Modify your Application**

In [5]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data = dataset_name,
    evaluators = [is_concise_enough],
    experiment_prefix = "gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-581a8bac' at:
https://smith.langchain.com/o/0d21f241-5c3d-4a95-af25-5842839689d3/datasets/32edf740-86da-490e-9e2f-d0faf30d18f9/compare?selectedSessions=7613b23c-2027-4e52-b1fe-d1390f0cfd0d




13it [00:39,  3.06s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I add tags to a Trace?,"To add tags to a trace in LangSmith, you can m...",None,You can add tags to a trace by annotating it e...,1,2.472741,9e30ed8a-08aa-4d87-9847-b0c67da920c4,c811ef57-3cd9-4f1d-ab52-0219e14112ca
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,Setting up tracing to LangSmith while using La...,1,3.655289,fee59a4e-f5d8-4f85-acfb-8433bebb6403,613d5b62-55d7-4d7c-a8b1-3cc575d3ab25
2,"{\n ""Question"": ""Is there a Javascript LangSmi...","Yes, there is a JavaScript/TypeScript LangSmit...",None,Yes there is a JavaScript LangSmith SDK,1,1.745807,d4b8c798-bafd-441e-8139-b76900b8cf9d,10516c7d-c6fa-4fb8-afeb-bf802aefa163
3,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,4.391612,070ce5f5-f180-4c04-8dfb-705f8201d79a,bbcc41a9-e951-4419-baf9-2ad15fc04936
4,What is LangSmith used for in three sentences?,"LangSmith is a platform for building, monitori...",None,LangSmith is a platform designed for the devel...,1,3.614561,0a303fe1-e0ec-4e4f-b6db-ecfcb08f8d6c,8109c062-e2f2-4d86-b10a-026c84198fa9
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,0,4.193476,0d731f1e-c7e9-4b10-bb80-4ab72cea0015,e792b4c0-71ba-45dc-aaa0-c2b7364893ff
6,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation. Af...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.624053,236f4c49-773a-4bf9-b31f-790d38b1761a,8fe8b472-621f-4e43-9641-6cf4d4f44aff
7,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,3.387206,6757b357-9aa0-48f9-96c5-b829c57662d9,04b48c38-3f8d-4c6b-9250-bed81fe734a0
8,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.741172,6cccd322-eec7-45b2-acd2-84ccfdb5e0ba,7d1916fb-3c9d-4e6d-91c6-afb9f6c3f15c
9,How do I pass metadata in with @traceable?,"To pass metadata with @traceable, you can deco...",None,You can pass metadata with the @traceable deco...,1,2.600502,7b6dddd4-f2bf-4aaa-b1fd-8098e73e4c21,cec35591-85a0-4bd5-8aed-68010d42bd85


### **Running over Different pieces of Data**
#### **Dataset Version**

In [ ]:
evaluate(
    target_function,
    data = client.list_examples(dataset_name = dataset_name, as_of = "initial dataset"),   # We use as_of to specify a version
    evaluators = [is_concise_enough],
    experiment_prefix = "initial dataset version"
)

#### **Dataset Split**

In [ ]:
evaluate(
    target_function,
    data = client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  # We pass in a list of Splits
    evaluators = [is_concise_enough],
    experiment_prefix = "Crucial Examples split"
)

#### **Specific Data Point**

In [ ]:
evaluate(
    target_function,
    data = client.list_examples(
        dataset_name = dataset_name, 
        example_ids = [   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "",
            ""
        ]
    ),
    evaluators = [is_concise_enough],
    experiment_prefix = "two specific example ids"
)

### **Other Parameters**
#### **Repetitions**

In [10]:
evaluate(
    target_function,
    data = dataset_name,
    evaluators = [is_concise_enough],
    experiment_prefix = "two repetitions",
    num_repetitions = 2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-bc16a69c' at:
https://smith.langchain.com/o/0d21f241-5c3d-4a95-af25-5842839689d3/datasets/32edf740-86da-490e-9e2f-d0faf30d18f9/compare?selectedSessions=9d1a36b0-c6d8-41b6-ad76-2656c471560e




26it [01:02,  2.40s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I add tags to a Trace?,You can add tags to a trace in LangSmith by ma...,None,You can add tags to a trace by annotating it e...,1,2.305957,9e30ed8a-08aa-4d87-9847-b0c67da920c4,22df45e7-1168-4ade-87b4-d565541a6a59
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,Setting up tracing to LangSmith while using La...,1,2.509118,fee59a4e-f5d8-4f85-acfb-8433bebb6403,9cfdfe5e-f9f0-403d-aa5d-4fdce34a37b5
2,"{\n ""Question"": ""Is there a Javascript LangSmi...","Yes, there is a JavaScript LangSmith SDK.",None,Yes there is a JavaScript LangSmith SDK,1,1.193783,d4b8c798-bafd-441e-8139-b76900b8cf9d,1b2d6ea3-7da8-4e5d-9d22-9dd680c30f3a
3,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,2.379929,070ce5f5-f180-4c04-8dfb-705f8201d79a,3b3f01ae-be2e-4a93-aaab-10cd0f0f8dac
4,What is LangSmith used for in three sentences?,"LangSmith is used for building, monitoring, an...",None,LangSmith is a platform designed for the devel...,1,2.113139,0a303fe1-e0ec-4e4f-b6db-ecfcb08f8d6c,97198fca-e413-4eb7-aa51-15e77fbe301c
5,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith with LangChain,...",None,To set up tracing to LangSmith while using Lan...,0,3.377070,0d731f1e-c7e9-4b10-bb80-4ab72cea0015,06e81fe6-62b5-4eba-9fe5-7a0cf824734c
6,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation.",None,"Yes, LangSmith supports offline evaluation thr...",1,0.983309,236f4c49-773a-4bf9-b31f-790d38b1761a,5eb2fb5b-2bc1-4e87-b591-fe8a06921f58
7,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.426840,6757b357-9aa0-48f9-96c5-b829c57662d9,09aa15ca-9596-4b75-827a-5ee72732fed0
8,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.824007,6cccd322-eec7-45b2-acd2-84ccfdb5e0ba,7ff944a7-4b2f-4532-831e-f29e04f01e63
9,How do I pass metadata in with @traceable?,To pass metadata with the @traceable decorator...,None,You can pass metadata with the @traceable deco...,1,2.675455,7b6dddd4-f2bf-4aaa-b1fd-8098e73e4c21,284d609f-cc1c-437b-8fa6-2ccf881307e3


#### **Concurrency**

In [11]:
evaluate(
    target_function,
    data = dataset_name,
    evaluators = [is_concise_enough],
    experiment_prefix = "concurrency",
    max_concurrency = 3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-0bc35ee6' at:
https://smith.langchain.com/o/0d21f241-5c3d-4a95-af25-5842839689d3/datasets/32edf740-86da-490e-9e2f-d0faf30d18f9/compare?selectedSessions=09b2dbdc-c0d6-46e1-a255-4ecdacc6ad24




13it [00:12,  1.07it/s]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,"{\n ""Question"": ""Is there a Javascript LangSmi...","Yes, LangSmith provides a JavaScript/TypeScrip...",None,Yes there is a JavaScript LangSmith SDK,1,1.653410,d4b8c798-bafd-441e-8139-b76900b8cf9d,ee0b4d20-c0ff-4d40-b5f3-096c66e409dc
1,How can I add tags to a Trace?,You can add tags to a trace in LangSmith by ma...,None,You can add tags to a trace by annotating it e...,1,2.526748,9e30ed8a-08aa-4d87-9847-b0c67da920c4,6e4fe300-9f0c-4e6e-b71b-ba8ff0016e54
2,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,Setting up tracing to LangSmith while using La...,1,3.041731,fee59a4e-f5d8-4f85-acfb-8433bebb6403,ecfaf91b-6409-40ba-b9f1-c1144e5b0534
3,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluations. Th...",None,"Yes, LangSmith supports online evaluation as a...",1,1.951814,070ce5f5-f180-4c04-8dfb-705f8201d79a,623c4d38-31cb-4e3f-9906-02b301258001
4,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation. Af...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.903085,236f4c49-773a-4bf9-b31f-790d38b1761a,766aa4ff-757c-4b4f-84e0-5ff1afbe82b6
5,What is LangSmith used for in three sentences?,"LangSmith is used for building, monitoring, an...",None,LangSmith is a platform designed for the devel...,1,3.468753,0a303fe1-e0ec-4e4f-b6db-ecfcb08f8d6c,96a456ae-0af6-48b9-8bea-8ac7b69f6a63
6,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.162031,6757b357-9aa0-48f9-96c5-b829c57662d9,e0c19959-e1f1-4b27-bb52-62ead49948c0
7,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith with LangChain,...",None,To set up tracing to LangSmith while using Lan...,0,4.873132,0d731f1e-c7e9-4b10-bb80-4ab72cea0015,b72ecd1f-21f1-43a7-b217-1d974f47e580
8,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.883381,6cccd322-eec7-45b2-acd2-84ccfdb5e0ba,6df0de03-fd14-4f38-8fa7-cddcf6ca0ecf
9,How do I pass metadata in with @traceable?,To pass metadata with the @traceable decorator...,None,You can pass metadata with the @traceable deco...,1,1.955788,7b6dddd4-f2bf-4aaa-b1fd-8098e73e4c21,25730d69-871b-4544-b56b-c964442cc3eb


#### **Metadata**

In [12]:
evaluate(
    target_function,
    data = dataset_name,
    evaluators = [is_concise_enough],
    experiment_prefix = "metadata added",
    metadata = {  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-d1b04c60' at:
https://smith.langchain.com/o/0d21f241-5c3d-4a95-af25-5842839689d3/datasets/32edf740-86da-490e-9e2f-d0faf30d18f9/compare?selectedSessions=5e4a4dcd-5db6-4ff4-9ce9-756d11d69668




13it [00:34,  2.67s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I add tags to a Trace?,"To add tags to a trace in LangSmith, you can m...",None,You can add tags to a trace by annotating it e...,1,5.270399,9e30ed8a-08aa-4d87-9847-b0c67da920c4,a70f0cea-f7c6-4120-91a9-5282a0605e63
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,Setting up tracing to LangSmith while using La...,1,2.312204,fee59a4e-f5d8-4f85-acfb-8433bebb6403,ca7e2f47-66f5-43c8-9b68-2aac05423fba
2,"{\n ""Question"": ""Is there a Javascript LangSmi...","Yes, there is a JavaScript LangSmith SDK.",None,Yes there is a JavaScript LangSmith SDK,1,1.346575,d4b8c798-bafd-441e-8139-b76900b8cf9d,d0e7ead4-5cfa-4111-9394-b713d306ec94
3,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.962226,070ce5f5-f180-4c04-8dfb-705f8201d79a,5783e62f-f9a6-47a0-b549-4afd14e78e67
4,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,3.688132,0a303fe1-e0ec-4e4f-b6db-ecfcb08f8d6c,03648e3a-e334-4e9c-8d04-7e28cd77645f
5,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith with LangChain,...",None,To set up tracing to LangSmith while using Lan...,0,3.599406,0d731f1e-c7e9-4b10-bb80-4ab72cea0015,dcfee21a-7d7d-4fe2-a1ca-2ae64a0b8dcc
6,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation.",None,"Yes, LangSmith supports offline evaluation thr...",1,1.629186,236f4c49-773a-4bf9-b31f-790d38b1761a,5e9b9a6f-b307-425f-943d-08788bc13292
7,How do I create user feedback with the LangSmi...,You can create user feedback with the LangSmit...,None,To create user feedback with the LangSmith SDK...,1,1.877928,6757b357-9aa0-48f9-96c5-b829c57662d9,200ea794-3f57-44c4-a585-192f0ff86dc9
8,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.401780,6cccd322-eec7-45b2-acd2-84ccfdb5e0ba,203bdc94-6253-4ff0-a2d1-8de4b220a48b
9,How do I pass metadata in with @traceable?,To pass metadata with the @traceable decorator...,None,You can pass metadata with the @traceable deco...,1,2.369613,7b6dddd4-f2bf-4aaa-b1fd-8098e73e4c21,a1f25048-5a50-4144-9273-459b775a80b0
